In [15]:
import pyspark
import os
import numpy as np
import pandas as pd
from scipy.stats import *
from pyspark.sql.functions import *
import pyspark.sql.functions as f
from pyspark.sql.types import *
from scipy.stats import norm
from scipy.stats import t

In [2]:
myConf = pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
.builder\
.master('local')\
.appName('myApp')\
.config(conf=myConf)\
.getOrCreate()

21/10/18 10:15:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/18 10:15:34 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
marks = [
    '김하나, English, 100',
    '김하나, Math, 80',
    '임하나, English, 70',
    '임하나, Math, 100',
    '김갑돌, English, 82.3',
    '김갑돌, Math, 98.5'
]

# 문제 1

In [4]:
col = ['name', 'subject', '_score']

_rdd = spark.sparkContext.parallelize(marks).map(lambda x : x.split(', '))
_df = spark.createDataFrame(_rdd, col)
_df = _df.withColumn('score', _df['_score'].cast('float')).drop('_score')

_df.printSchema()
_df.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- score: float (nullable = true)

+------+-------+-----+
|  name|subject|score|
+------+-------+-----+
|김하나|English|100.0|
|김하나|   Math| 80.0|
|임하나|English| 70.0|
|임하나|   Math|100.0|
|김갑돌|English| 82.3|
|김갑돌|   Math| 98.5|
+------+-------+-----+



# 문제 2

In [6]:
# 전체 데이터의 평균과 표준편차를 알아야 한다.
# zscore 데이터 컬럼 생성
scores = np.array(_rdd.map(lambda x : float(x[2])).collect())
xbar = scores.mean()
sig = scores.std()
xbar, sig

(88.46666666666665, 11.672141572517397)

In [13]:
udf_zscore = udf(lambda x : float((x - xbar) / sig), FloatType())
_df = _df.withColumn('zscore', udf_zscore(_df.score))
_df.printSchema()
_df.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- score: float (nullable = true)
 |-- zscore: float (nullable = true)

+------+-------+-----+-----------+
|  name|subject|score|     zscore|
+------+-------+-----+-----------+
|김하나|English|100.0| 0.98810774|
|김하나|   Math| 80.0|-0.72537386|
|임하나|English| 70.0| -1.5821147|
|임하나|   Math|100.0| 0.98810774|
|김갑돌|English| 82.3|-0.52832323|
|김갑돌|   Math| 98.5|  0.8595966|
+------+-------+-----+-----------+



# 문제 3 (N이 작지 않다고 봤을 때

In [14]:
# 각 row의 zscore에 대한 cdf를 구하라는 건가요??
udf_cdf = udf(lambda x : float(norm.cdf(x, loc = 0, scale = 1)), FloatType())
_df = _df.withColumn('cdf', udf_cdf(_df.zscore))
_df.printSchema()
_df.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- score: float (nullable = true)
 |-- zscore: float (nullable = true)
 |-- cdf: float (nullable = true)

+------+-------+-----+-----------+-----------+
|  name|subject|score|     zscore|        cdf|
+------+-------+-----+-----------+-----------+
|김하나|English|100.0| 0.98810774|  0.8384501|
|김하나|   Math| 80.0|-0.72537386| 0.23411134|
|임하나|English| 70.0| -1.5821147|0.056811694|
|임하나|   Math|100.0| 0.98810774|  0.8384501|
|김갑돌|English| 82.3|-0.52832323|  0.2986375|
|김갑돌|   Math| 98.5|  0.8595966|  0.8049943|
+------+-------+-----+-----------+-----------+



# 문제 3 (N이 작다고 봤을 때 (t score)

In [16]:
# 각 row의 zscore에 대한 cdf를 구하라는 건가요??
udf_cdf = udf(lambda x : float(t.cdf(x, 1, loc = 0, scale = 1)), FloatType())
_df = _df.withColumn('t.cdf', udf_cdf(_df.zscore))
_df.printSchema()
_df.show()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- score: float (nullable = true)
 |-- zscore: float (nullable = true)
 |-- cdf: float (nullable = true)
 |-- t.cdf: float (nullable = true)



+------+-------+-----+-----------+-----------+----------+
|  name|subject|score|     zscore|        cdf|     t.cdf|
+------+-------+-----+-----------+-----------+----------+
|김하나|English|100.0| 0.98810774|  0.8384501|  0.748096|
|김하나|   Math| 80.0|-0.72537386| 0.23411134|0.30024362|
|임하나|English| 70.0| -1.5821147|0.056811694| 0.1794198|
|임하나|   Math|100.0| 0.98810774|  0.8384501|  0.748096|
|김갑돌|English| 82.3|-0.52832323|  0.2986375|0.34528592|
|김갑돌|   Math| 98.5|  0.8595966|  0.8049943|0.72601247|
+------+-------+-----+-----------+-----------+----------+

